<a href="https://colab.research.google.com/github/alendoko/bina/blob/main/LB4/DATA/Python_JUPYTER_Data_Analysis_SteyByStep_Melbourne_Houseprice2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jupiter Notebook, welches die Solarenergie-Entwicklung in der Schweiz der letzten 10 Jahre analysiert

## Daten laden, aufbereiten und anreichern

In [1]:
import pandas as pd
import plotly.express as px

file_path_energy_data = '../data/energy_sources/elektrizitaetbilanz_jahreswerte.csv'
energy_data = pd.read_csv(file_path_energy_data, delimiter=';')

# Nur Spalten behalten, die mit 'Erzeugung_' beginnen oder 'Jahr' enthalten
columns_to_keep = ['Jahr'] + [col for col in energy_data.columns if col.startswith('Erzeugung_')]
energy_data = energy_data[columns_to_keep]


energy_data['Wasserkraft'] = (energy_data['Erzeugung_laufwerk_GWh'] + energy_data['Erzeugung_speicherwerk_GWh'])
energy_data['Andere_erneuerbare_Energien_GWh'] = (energy_data['Erzeugung_holz_GWh'] + energy_data['Erzeugung_biogas_GWh'] + energy_data['Erzeugung_wind_GWh'])

# Daten filtern: Nur Werte ab 2015
energy_data = energy_data[energy_data['Jahr'] >= 2015]

# Sicherstellen, dass alle Jahre als Kategorie behandelt werden
energy_data['Jahr'] = energy_data['Jahr'].astype(str)

# Spalten für die melt-Operation angeben und Namen ändern
columns_to_melt = ['Wasserkraft', 'Erzeugung_kernkraftwerk_GWh', 'Erzeugung_andere_total_GWh', 'Erzeugung_photovoltaik_GWh', 'Andere_erneuerbare_Energien_GWh']

# Perform the melt operation
energy_data_melted = energy_data.melt(
    id_vars=['Jahr'], 
    value_vars=columns_to_melt, 
    var_name='Energiequelle', 
    value_name='Wert'
)

# Spalten umbenennen
energy_data_melted['Energiequelle'] = energy_data_melted['Energiequelle'].replace({
    'Erzeugung_kernkraftwerk_GWh': 'Kernkraft',
    'Erzeugung_andere_total_GWh': 'Thermische Kraft',
    'Erzeugung_photovoltaik_GWh': 'Photovoltaik',
    'Andere_erneuerbare_Energien_GWh': 'Andere erneuerbare Energien'
})


## Flächenchart erstellen, der Energieproduktion nach Quellen in der Schweiz

In [2]:
# Gestapeltes Flächenchart erstellen
fig = px.area(
    energy_data_melted,
    x='Jahr',
    y='Wert',
    color='Energiequelle',
    title='Energieproduktion nach Quellen in der Schweiz',
    labels={'Wert': 'Energieproduktion in GWh', 'Jahr': 'Jahr', 'Energiequelle': 'Energiequelle'}
)

# Chart anzeigen
fig.show()

# Speichern des Diagramms als HTML-Datei
fig.write_html('../docs/assets/diagramme/solar_vs_other_energy_sources.html')